<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/instrGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction Generation

## Basic includes

In [ ]:
!pip install ipython-autotime
%load_ext autotime
!pip install word2vec
!pip install ray[tune]

## Connect colab and set paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

rootDir = '/content/drive/MyDrive/'

TIMESTAMP = '2022_03_19'

dataPath = rootDir + 'TP2/Datasets/Recipe1M/' + TIMESTAMP
tarPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP


import sys
sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM')


In [ ]:
# local exec
tarPath = 'D:/02_Studium/SBX/mad-recime/data/2022_03_19'

import os
sys.path.append(os.getcwd())

## Basic includes

In [ ]:
import pandas as pd
import numpy as np

import torch

# Model
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
from torchsummary import summary

# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# hyperparameter tuning
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [ ]:
import src.instGen as instGen
import src.preProc as preProc
import importlib

## Seed

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

## Training execution
mixture of 
* https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
* https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html
* https://stackoverflow.com/questions/67295494/correct-validation-loss-in-pytorch


### Set device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print(torch.cuda.get_device_name(0))

### Set hyperparams

In [ ]:
importlib.reload(instGen)

hyperParams = instGen.HyperParams(epochs=1, batchSize=2, hiddenDim=10, numLayers=1, embeddingDim=100)
print(hyperParams)

### Set dataset

In [ ]:
importlib.reload(instGen)
importlib.reload(preProc)

instSet = instGen.InstructionSet(tarPath, setSize=2)

# split data set
trainNum = int(hyperParams.ratio[0] * len(instSet))
valNum = int(hyperParams.ratio[1] * len(instSet))


splitSet = random_split(
    instSet, [trainNum, valNum], generator=torch.Generator().manual_seed(0))
    
splitSet = {'train': splitSet[0], 'val': splitSet[1]}

testSet = instGen.getPreProcData(tarPath, range(-1,0))

In [ ]:
wordCount = pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_counts).sort_values())
wordCount

In [ ]:
pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_index))

### Set model

In [ ]:
importlib.reload(instGen)

model = instGen.EmbedLSTM(hyperParams, instSet, device)
model.to(device)
print(model)


### Train

In [ ]:
importlib.reload(instGen)

instGen.train(splitSet, model, hyperParams, device)

### Test

In [ ]:
importlib.reload(instGen)

idx = np.random.randint(0, len(testSet))
inputIngs = testSet['ingredient'].values[idx]
inputTitle = testSet['title'].values[idx].split()
seq = instGen.testInputPrep(inputIngs + inputTitle, instSet.tokenizer)
print(inputIngs)
print(inputTitle)
print(seq)
instGen.sample(model, instSet, 300, device, initial=seq).split(instSet.delimiter)
# todo: get avg length of instructions

## Save model

In [ ]:
torch.save(model.state_dict(),
           rootDir + 'weights/instGenerator_model.pt')


# Tensorboard visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/runs/instTrainer